In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, t

# Parametry
sample_sizes = [20, 50, 100]
t_dfs = [2, 5, 10, 30]
K = 500
alpha = 0.05

# Funkcja testu Kołmogorowa
def kolmogorov_test(data):
    mu = np.mean(data)
    sigma = np.std(data, ddof=1)
    _, p_value = kstest(data, 'norm', args=(mu, sigma))
    return p_value

# Symulacja mocy
def simulate_power(use_pit=False):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in t_dfs:
            rejections = 0
            for _ in range(K):
                sample = t.rvs(df, size=n)
                if use_pit:
                    sample = t.cdf(sample, df)
                p = kolmogorov_test(sample)
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Uruchom testy
results_classic = simulate_power(use_pit=False)
results_pit = simulate_power(use_pit=True)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(t_dfs, results_classic[n], marker='o', label=f'n={n} (classic)')
    plt.plot(t_dfs, results_pit[n], marker='x', linestyle='--', label=f'n={n} (PIT)')

plt.title('Moc testu Kołmogorowa dla danych z rozkładu t-Studenta')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, chi2

# Parametry
sample_sizes = [20, 50, 100]
chi2_dfs = [2, 4, 8, 16]
K = 500
alpha = 0.05

# Funkcja testu Kołmogorowa
def kolmogorov_test(data):
    mu = np.mean(data)
    sigma = np.std(data, ddof=1)
    _, p_value = kstest(data, 'norm', args=(mu, sigma))
    return p_value

# Symulacja mocy testu Kołmogorowa dla rozkładu chi2
def simulate_power_chi2(sample_sizes, dfs, K=500, alpha=0.05, use_pit=False):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in dfs:
            rejections = 0
            for _ in range(K):
                data = chi2.rvs(df, size=n)
                if use_pit:
                    data = chi2.cdf(data, df)
                p = kolmogorov_test(data)
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Symulacje
results_classic = simulate_power_chi2(sample_sizes, chi2_dfs, K=K, alpha=alpha, use_pit=False)
results_pit = simulate_power_chi2(sample_sizes, chi2_dfs, K=K, alpha=alpha, use_pit=True)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(chi2_dfs, results_classic[n], marker='o', label=f'n={n} (classic)')
    plt.plot(chi2_dfs, results_pit[n], marker='x', linestyle='--', label=f'n={n} (PIT)')

plt.title('Moc testu Kołmogorowa dla danych z rozkładu chi-kwadrat')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, t

# Parametry
sample_sizes = [20, 50, 100]
t_dfs = [2, 5, 10, 30]
K = 500
alpha = 0.05

# Funkcja testu Kołmogorowa
def kolmogorov_test(data):
    mu = np.mean(data)
    sigma = np.std(data, ddof=1)
    _, p_value = kstest(data, 'norm', args=(mu, sigma))
    return p_value

# Symulacja mocy
def simulate_power():
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in t_dfs:
            rejections = 0
            for _ in range(K):
                sample = t.rvs(df, size=n)
                p = kolmogorov_test(sample)
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Testy
results_classic = simulate_power()

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(t_dfs, results_classic[n], marker='o', label=f'n={n}')

plt.title('Moc testu Kołmogorowa dla danych z rozkładu t-Studenta (bez PIT)')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Wnioski:
# - Im mniejszy df, tym większe odstępstwo od normalności → większa moc testu
# - Im większa próbka, tym większa czułość testu (wyższa moc)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, chi2

# Parametry
sample_sizes = [20, 50, 100]
chi2_dfs = [2, 4, 8, 16]
K = 500
alpha = 0.05

# Funkcja testu Kołmogorowa
def kolmogorov_test(data):
    mu = np.mean(data)
    sigma = np.std(data, ddof=1)
    _, p_value = kstest(data, 'norm', args=(mu, sigma))
    return p_value

# Symulacja mocy testu Kołmogorowa dla rozkładu chi2 (bez PIT)
def simulate_power_chi2(sample_sizes, dfs, K=500, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in dfs:
            rejections = 0
            for _ in range(K):
                data = chi2.rvs(df, size=n)
                p = kolmogorov_test(data)
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Symulacje (tylko klasyczne)
results_classic = simulate_power_chi2(sample_sizes, chi2_dfs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(chi2_dfs, results_classic[n], marker='o', label=f'n={n}')

plt.title('Moc testu Kołmogorowa dla danych z rozkładu chi-kwadrat (bez PIT)')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Wnioski:
# - Im większa liczba df, tym bardziej rozkład zbliża się do normalnego → moc testu spada
# - Im większa liczność próbki, tym większa czułość testu (wyższa moc)
# - PIT nie ma zastosowania w tym kontekście, bo test porównuje z N(μ,σ), a nie U(0,1)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, norm

# Parametry
sample_sizes = [20, 50, 100]
std_devs = [0.5, 1.0, 1.5, 2.0]
K = 500
alpha = 0.05

# Test Kołmogorowa bez estymacji (względem znanego N(0, std))
def kolmogorov_test_known_std(data, std):
    _, p_value = kstest(data, 'norm', args=(0, std))
    return p_value

# Symulacja mocy testu Kołmogorowa (dane z N(0, σ))
def simulate_power_normal_sigma_known(sample_sizes, std_devs, K=500, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for std in std_devs:
            rejections = 0
            for _ in range(K):
                data = norm.rvs(loc=0, scale=std, size=n)
                p = kolmogorov_test_known_std(data, std)
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Uruchom symulacje
results_correct = simulate_power_normal_sigma_known(sample_sizes, std_devs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(std_devs, results_correct[n], marker='o', label=f'n={n}')

plt.title('Moc testu Kołmogorowa dla danych z N(0, σ²) (test względem znanego N(0, σ))')
plt.xlabel('Odchylenie standardowe (σ)')
plt.ylabel('Moc testu')
plt.ylim(0, 0.1)  # zakładamy moc blisko 0.05
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Oczekiwany efekt:
# Moc ≈ 0.05 niezależnie od σ i n — bo H₀ jest prawdziwa i test jest dobrze skonfigurowany.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, gamma, norminvgauss

# Parametry
sample_sizes = [20, 50, 100]
K = 500
alpha = 0.05

# Parametry rozkładów
gamma_params = [(2, 2), (5, 1)]         
nig_params = [(1, 0, 0, 1), (2, 0, 0, 1)]  

# --- BŁĄD I RODZAJU ---

def error_type1(test_dist, dist_name, param_grid, sample_sizes, K=500, alpha=0.05):
    results_classic = {n: [] for n in sample_sizes}
    results_pit = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in param_grid:
            reject_c = 0
            reject_p = 0
            for _ in range(K):
                data = test_dist.rvs(*params, size=n)
                p1 = kstest(data, dist_name, args=params)[1]
                if p1 < alpha:
                    reject_c += 1
                uniform_data = test_dist.cdf(data, *params)
                p2 = kstest(uniform_data, 'uniform')[1]
                if p2 < alpha:
                    reject_p += 1
            results_classic[n].append(reject_c / K)
            results_pit[n].append(reject_p / K)
    return results_classic, results_pit

# Symulacje błędu I rodzaju
error_gamma_classic, error_gamma_pit = error_type1(gamma, 'gamma', gamma_params, sample_sizes, K, alpha)
error_nig_classic, error_nig_pit = error_type1(norminvgauss, 'norminvgauss', nig_params, sample_sizes, K, alpha)

# --- MOC TESTU KOŁMOGOROWA ---

def simulate_power_vs_normal(test_dist, dist_name, param_grid, sample_sizes, K=500, alpha=0.05):
    results_classic = {n: [] for n in sample_sizes}
    results_pit = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in param_grid:
            reject_c = 0
            reject_p = 0
            for _ in range(K):
                data = test_dist.rvs(*params, size=n)
                mu = np.mean(data)
                sigma = np.std(data, ddof=1)
                p1 = kstest(data, 'norm', args=(mu, sigma))[1]
                if p1 < alpha:
                    reject_c += 1
                u_data = test_dist.cdf(data, *params)
                p2 = kstest(u_data, 'uniform')[1]
                if p2 < alpha:
                    reject_p += 1
            results_classic[n].append(reject_c / K)
            results_pit[n].append(reject_p / K)
    return results_classic, results_pit

# Symulacje mocy testu (czyli trafne odrzucenie H₀: normalność)
power_gamma_classic, power_gamma_pit = simulate_power_vs_normal(gamma, 'gamma', gamma_params, sample_sizes, K, alpha)
power_nig_classic, power_nig_pit = simulate_power_vs_normal(norminvgauss, 'norminvgauss', nig_params, sample_sizes, K, alpha)

# --- WYKRESY ---

def plot_results(results_c, results_p, param_labels, sample_sizes, title, ylabel='Błąd I rodzaju / Moc testu'):
    plt.figure(figsize=(10, 6))
    for n in sample_sizes:
        plt.plot(param_labels, results_c[n], marker='o', label=f'n={n} (classic)')
        plt.plot(param_labels, results_p[n], marker='x', linestyle='--', label=f'n={n} (PIT)')
    plt.axhline(0.05, color='gray', linestyle='dotted', label='Poziom istotności (0.05)')
    plt.title(title)
    plt.xlabel('Parametry rozkładu')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Wykresy błędu I rodzaju
plot_results(error_gamma_classic, error_gamma_pit,
             param_labels=[str(p) for p in gamma_params],
             sample_sizes=sample_sizes,
             title='Błąd I rodzaju – test Kołmogorowa dla rozkładu gamma')

plot_results(error_nig_classic, error_nig_pit,
             param_labels=[str(p) for p in nig_params],
             sample_sizes=sample_sizes,
             title='Błąd I rodzaju – test Kołmogorowa dla rozkładu NIG')

# Wykresy mocy testu
plot_results(power_gamma_classic, power_gamma_pit,
             param_labels=[str(p) for p in gamma_params],
             sample_sizes=sample_sizes,
             title='Moc testu Kołmogorowa – dane z rozkładu gamma (vs H₀: normalność)',
             ylabel='Moc testu')

plot_results(power_nig_classic, power_nig_pit,
             param_labels=[str(p) for p in nig_params],
             sample_sizes=sample_sizes,
             title='Moc testu Kołmogorowa – dane z rozkładu NIG (vs H₀: normalność)',
             ylabel='Moc testu')


In [ ]:
from statsmodels.stats.diagnostic import lilliefors
from scipy.stats import t
import numpy as np
import matplotlib.pyplot as plt

# Parametry
sample_sizes = [20, 50, 100]
t_dfs = [2, 5, 10, 30]
K = 500
alpha = 0.05

# Funkcja: moc testu Lillieforsa dla rozkładu t-Studenta
def simulate_lilliefors_power_tstudent(sample_sizes, dfs, K=500, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in dfs:
            rejections = 0
            for _ in range(K):
                data = t.rvs(df, size=n)
                stat, p = lilliefors(data, dist='norm')
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Uruchom testy
lilliefors_results = simulate_lilliefors_power_tstudent(sample_sizes, t_dfs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(t_dfs, lilliefors_results[n], marker='o', label=f'n={n}')

plt.title('Moc testu Lillieforsa dla danych z rozkładu t-Studenta')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2
from statsmodels.stats.diagnostic import lilliefors

# Parametry
sample_sizes = [20, 50, 100]
chi2_dfs = [2, 4, 8, 16]
K = 500
alpha = 0.05

# Funkcja: moc testu Lillieforsa dla rozkładu chi-kwadrat
def simulate_lilliefors_power_chi2(sample_sizes, dfs, K=500, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in dfs:
            rejections = 0
            for _ in range(K):
                data = chi2.rvs(df, size=n)
                stat, p = lilliefors(data, dist='norm')
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Uruchom testy
lilliefors_results_chi2 = simulate_lilliefors_power_chi2(sample_sizes, chi2_dfs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(chi2_dfs, lilliefors_results_chi2[n], marker='o', label=f'n={n}')

plt.title('Moc testu Lillieforsa dla danych z rozkładu chi-kwadrat')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from statsmodels.stats.diagnostic import lilliefors

# Parametry
sample_sizes = [20, 50, 100]
std_devs = [0.5, 1.0, 1.5, 2.0]
K = 500
alpha = 0.05

# Funkcja: moc testu Lillieforsa dla danych z rozkładu normalnego N(0, σ²)
def simulate_lilliefors_power_normal_std(sample_sizes, std_devs, K=500, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for std in std_devs:
            rejections = 0
            for _ in range(K):
                data = norm.rvs(loc=0, scale=std, size=n)
                stat, p = lilliefors(data, dist='norm')
                if p < alpha:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Uruchom testy
lilliefors_power_normal_std = simulate_lilliefors_power_normal_std(sample_sizes, std_devs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(std_devs, lilliefors_power_normal_std[n], marker='o', label=f'n={n}')

plt.title('Moc testu Lillieforsa dla danych z rozkładu normalnego N(0, σ²)')
plt.xlabel('Odchylenie standardowe (σ)')
plt.ylabel('Moc testu')
plt.ylim(0.0, 0.1)  # 👈 zawężenie osi Y dla lepszej widoczności
plt.axhline(0.05, color='gray', linestyle='dotted', label='Poziom istotności (0.05)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma, kstest
from statsmodels.stats.diagnostic import lilliefors

# Parametry
sample_sizes = [20, 50, 100]
gamma_params = [(2, 2), (5, 1), (9, 0.5)]
K = 100
alpha = 0.05

# Moc testu Lillieforsa
def simulate_lilliefors_power_custom(dist, param_grid, sample_sizes, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in param_grid:
            rejections = 0
            for _ in range(K):
                data = dist.rvs(*params, size=n)
                stat, p = lilliefors(data, dist='norm')
                if p < alpha:
                    rejections += 1
            results[n].append(rejections / K)
    return results

# Błąd I rodzaju z PIT
def simulate_pit_error_custom(dist, param_grid, sample_sizes, K=100, alpha=0.05):
    results_pit = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in param_grid:
            rejections = 0
            for _ in range(K):
                data = dist.rvs(*params, size=n)
                u_data = dist.cdf(data, *params)
                _, p = kstest(u_data, 'uniform')
                if p < alpha:
                    rejections += 1
            results_pit[n].append(rejections / K)
    return results_pit

# Symulacje
lilliefors_gamma_power = simulate_lilliefors_power_custom(gamma, gamma_params, sample_sizes, K, alpha)
pit_error_gamma = simulate_pit_error_custom(gamma, gamma_params, sample_sizes, K, alpha)

# Wykres
def plot_lilliefors_vs_pit(power_results, pit_results, param_labels, sample_sizes, title, ylabel='Moc testu / Błąd I rodzaju'):
    plt.figure(figsize=(10, 6))
    for n in sample_sizes:
        plt.plot(param_labels, power_results[n], marker='o', label=f'n={n} (Lilliefors)')
        plt.plot(param_labels, pit_results[n], marker='x', linestyle='--', label=f'n={n} (PIT)')
    plt.axhline(0.05, color='gray', linestyle='dotted', label='Poziom istotności (0.05)')
    plt.title(title)
    plt.xlabel('Parametry rozkładu')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Wywołanie wykresu
plot_lilliefors_vs_pit(
    lilliefors_gamma_power,
    pit_error_gamma,
    param_labels=[str(p) for p in gamma_params],
    sample_sizes=sample_sizes,
    title='Moc testu Lillieforsa i PIT – dane z rozkładu gamma'
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, gamma, norminvgauss

# Parametry
sample_sizes = [20, 50, 100]
gamma_params = [(2, 2), (5, 1), (9, 0.5)]
nig_params = [(1, 0, 0, 1), (2, 0, 0, 1), (5, 0, 0, 1)]
alpha = 0.05

# === Funkcja: błąd I rodzaju klasyczny + PIT ===
def simulate_type1_error_kolmogorov(dist, param_grid, sample_sizes, K=100, alpha=0.05):
    results_classic = {n: [] for n in sample_sizes}
    results_pit = {n: [] for n in sample_sizes}

    for n in sample_sizes:
        for params in param_grid:
            reject_c = 0
            reject_p = 0
            for _ in range(K):
                data = dist.rvs(*params, size=n)

                # Klasyczny test
                p1 = kstest(data, dist.name, args=params)[1]
                if p1 < alpha:
                    reject_c += 1

                # PIT
                u_data = dist.cdf(data, *params)
                p2 = kstest(u_data, 'uniform')[1]
                if p2 < alpha:
                    reject_p += 1

            results_classic[n].append(reject_c / K)
            results_pit[n].append(reject_p / K)
    
    return results_classic, results_pit

# === Symulacja dla GAMMA ===
type1_gamma_classic, type1_gamma_pit = simulate_type1_error_kolmogorov(
    gamma, gamma_params, sample_sizes, K=100, alpha=alpha
)

# === Symulacja dla NIG (ograniczona) ===
type1_nig_classic, type1_nig_pit = simulate_type1_error_kolmogorov(
    norminvgauss, nig_params, sample_sizes, K=10, alpha=alpha
)

# === Wykresy porównawcze ===
def plot_type1_error_comparison(results_c, results_p, param_labels, sample_sizes, title):
    plt.figure(figsize=(10, 6))
    for n in sample_sizes:
        plt.plot(param_labels, results_c[n], marker='o', label=f'n={n} (classic)')
        plt.plot(param_labels, results_p[n], marker='x', linestyle='--', label=f'n={n} (PIT)')
    plt.axhline(0.05, color='gray', linestyle='dotted', label='Poziom istotności (0.05)')
    plt.title(title)
    plt.xlabel('Parametry rozkładu')
    plt.ylabel('Błąd I rodzaju')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# === Wykres: GAMMA ===
plot_type1_error_comparison(
    type1_gamma_classic, type1_gamma_pit,
    param_labels=[str(p) for p in gamma_params],
    sample_sizes=sample_sizes,
    title='Błąd I rodzaju – test Kołmogorowa dla rozkładu gamma'
)

# === Wykres: NIG ===
plot_type1_error_comparison(
    type1_nig_classic, type1_nig_pit,
    param_labels=[str(p) for p in nig_params],
    sample_sizes=sample_sizes,
    title='Błąd I rodzaju – test Kołmogorowa dla rozkładu NIG (K=10)'
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kstest, gamma, norm
from statsmodels.stats.diagnostic import lilliefors

# Parametry
sample_sizes = [20, 50, 100]
gamma_params = [(2, 2), (5, 1), (9, 0.5)]
K = 500
alpha = 0.05

# --- Moc testu Lillieforsa ---
def simulate_lilliefors_power_custom(dist, param_grid, sample_sizes, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in param_grid:
            rejections = 0
            for _ in range(K):
                data = dist.rvs(*params, size=n)
                stat, p = lilliefors(data, dist='norm')
                if p < alpha:
                    rejections += 1
            results[n].append(rejections / K)
    return results

# --- PIT: test jednostajności po przekształceniu przez CDF ---
def simulate_pit_error_custom(dist, param_grid, sample_sizes, K=100, alpha=0.05):
    results_pit = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in param_grid:
            rejections = 0
            for _ in range(K):
                data = dist.rvs(*params, size=n)
                u_data = dist.cdf(data, *params)
                _, p = kstest(u_data, 'uniform')
                if p < alpha:
                    rejections += 1
            results_pit[n].append(rejections / K)
    return results_pit

# --- Błąd I rodzaju testu Lillieforsa (na danych z N(0,1)) ---
def simulate_lilliefors_type1_error(dist, param_grid, sample_sizes, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for _ in param_grid:
            rejections = 0
            for _ in range(K):
                data = dist.rvs(loc=0, scale=1, size=n)
                stat, p = lilliefors(data, dist='norm')
                if p < alpha:
                    rejections += 1
            results[n].append(rejections / K)
    return results

# --- Symulacje ---
lilliefors_gamma_power = simulate_lilliefors_power_custom(gamma, gamma_params, sample_sizes, K, alpha)
pit_error_gamma = simulate_pit_error_custom(gamma, gamma_params, sample_sizes, K, alpha)
lilliefors_type1_gamma = simulate_lilliefors_type1_error(norm, gamma_params, sample_sizes, K, alpha)

# --- Wykres mocy testu + PIT ---
def plot_lilliefors_vs_pit(power_results, pit_results, param_labels, sample_sizes, title, ylabel='Moc testu / Błąd I rodzaju'):
    plt.figure(figsize=(10, 6))
    for n in sample_sizes:
        plt.plot(param_labels, power_results[n], marker='o', label=f'n={n} (Lilliefors)')
        plt.plot(param_labels, pit_results[n], marker='x', linestyle='--', label=f'n={n} (PIT)')
    plt.axhline(0.05, color='gray', linestyle='dotted', label='Poziom istotności (0.05)')
    plt.title(title)
    plt.xlabel('Parametry rozkładu')
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

plot_lilliefors_vs_pit(
    lilliefors_gamma_power,
    pit_error_gamma,
    param_labels=[str(p) for p in gamma_params],
    sample_sizes=sample_sizes,
    title='Moc testu Lillieforsa i PIT – dane z rozkładu gamma'
)

# --- Wykres błędu I rodzaju (tylko Lilliefors) ---
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot([str(p) for p in gamma_params], lilliefors_type1_gamma[n], marker='o', label=f'n={n}')
plt.axhline(0.05, color='gray', linestyle='dotted', label='Poziom istotności (0.05)')
plt.title('Błąd I rodzaju testu Lillieforsa – dane z rozkładu normalnego')
plt.xlabel('Parametry gamma (etykiety tylko dla porównania)')
plt.ylabel('Błąd I rodzaju')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norminvgauss
from statsmodels.stats.diagnostic import lilliefors

# Parametry
sample_sizes = [20, 50, 100]
nig_params = [(1, 0, 0, 1), (2, 0, 0, 1), (5, 0, 0, 1)]  # (alpha, beta, loc, scale)
K = 100
alpha = 0.05

# Funkcja: moc testu Lillieforsa dla danych z rozkładu NIG
def simulate_lilliefors_power_nig(sample_sizes, nig_param_grid, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in nig_param_grid:
            rejections = 0
            for _ in range(K):
                data = norminvgauss.rvs(*params, size=n)
                stat, p = lilliefors(data, dist='norm')
                if p < alpha:
                    rejections += 1
            results[n].append(rejections / K)
    return results

# Symulacja
lilliefors_nig_power = simulate_lilliefors_power_nig(sample_sizes, nig_params, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot([str(p) for p in nig_params], lilliefors_nig_power[n], marker='o', label=f'n={n}')
plt.title('Moc testu Lillieforsa dla danych z rozkładu NIG')
plt.xlabel('Parametry rozkładu NIG')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t, anderson

# Parametry
sample_sizes = [20, 50, 100]
t_dfs = [2, 5, 10, 30]
K = 100  # liczba symulacji
alpha = 0.05

# Funkcja: moc testu Andersona-Darlinga dla danych z t-Studenta
def simulate_ad_power_tstudent(sample_sizes, dfs, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in dfs:
            rejections = 0
            for _ in range(K):
                data = t.rvs(df, size=n)
                result = anderson(data, dist='norm')
                # Porównanie ze statystyką krytyczną dla poziomu 5%
                if result.statistic > result.critical_values[2]:
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Symulacja
ad_results_t = simulate_ad_power_tstudent(sample_sizes, t_dfs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(t_dfs, ad_results_t[n], marker='o', label=f'n={n}')
plt.title('Moc testu Andersona-Darlinga dla danych z rozkładu t-Studenta')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2, anderson

# Parametry
sample_sizes = [20, 50, 100]
chi2_dfs = [2, 4, 8, 16]
K = 100
alpha = 0.05

# Funkcja: moc testu Andersona-Darlinga dla danych z rozkładu chi²
def simulate_ad_power_chi2(sample_sizes, dfs, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for df in dfs:
            rejections = 0
            for _ in range(K):
                data = chi2.rvs(df, size=n)
                result = anderson(data, dist='norm')
                if result.statistic > result.critical_values[2]:  # Poziom istotności 5%
                    rejections += 1
            power = rejections / K
            results[n].append(power)
    return results

# Symulacja
ad_results_chi2 = simulate_ad_power_chi2(sample_sizes, chi2_dfs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(chi2_dfs, ad_results_chi2[n], marker='o', label=f'n={n}')
plt.title('Moc testu Andersona-Darlinga dla danych z rozkładu chi-kwadrat')
plt.xlabel('Stopnie swobody (df)')
plt.ylabel('Moc testu')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, anderson

# Parametry
sample_sizes = [20, 50, 100]
std_devs = [0.5, 1.0, 1.5, 2.0]
K = 100
alpha = 0.05

# Funkcja: błąd I rodzaju testu Andersona-Darlinga dla N(0, σ²)
def simulate_ad_type1_error_normal(sample_sizes, std_devs, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for std in std_devs:
            rejections = 0
            for _ in range(K):
                data = norm.rvs(loc=0, scale=std, size=n)
                result = anderson(data, dist='norm')
                if result.statistic > result.critical_values[2]:
                    rejections += 1
            error_rate = rejections / K
            results[n].append(error_rate)
    return results

# Symulacja
ad_type1_results = simulate_ad_type1_error_normal(sample_sizes, std_devs, K=K, alpha=alpha)

# Wykres
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot(std_devs, ad_type1_results[n], marker='o', label=f'n={n}')
plt.axhline(0.05, color='gray', linestyle='dotted', label='Poziom istotności (0.05)')
plt.title('Błąd I rodzaju testu Andersona-Darlinga – dane z N(0, σ²)')
plt.xlabel('Odchylenie standardowe (σ)')
plt.ylabel('Błąd I rodzaju')
plt.ylim(0.0, 0.1)  # 👈 zawężona oś Y dla lepszej widoczności
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gamma, norminvgauss, anderson

# Parametry
sample_sizes = [20, 50, 100]
gamma_params = [(2, 2), (5, 1), (9, 0.5)]
nig_params = [(1, 0, 0, 1), (2, 0, 0, 1), (5, 0, 0, 1)]
K = 100
alpha = 0.05

# Funkcja: moc testu Andersona-Darlinga dla dowolnego rozkładu
def simulate_ad_power_custom(dist, param_grid, sample_sizes, K=100, alpha=0.05):
    results = {n: [] for n in sample_sizes}
    for n in sample_sizes:
        for params in param_grid:
            rejections = 0
            for _ in range(K):
                data = dist.rvs(*params, size=n)
                result = anderson(data, dist='norm')
                if result.statistic > result.critical_values[2]:
                    rejections += 1
            results[n].append(rejections / K)
    return results

# Symulacja mocy
ad_power_gamma = simulate_ad_power_custom(gamma, gamma_params, sample_sizes, K=K, alpha=alpha)
ad_power_nig = simulate_ad_power_custom(norminvgauss, nig_params, sample_sizes, K=K, alpha=alpha)

# Wykres: GAMMA
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot([str(p) for p in gamma_params], ad_power_gamma[n], marker='o', label=f'n={n}')
plt.title('Moc testu Andersona-Darlinga – dane z rozkładu gamma')
plt.xlabel('Parametry rozkładu gamma (shape, scale)')
plt.ylabel('Moc testu')
plt.ylim(0.0, 1.0)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Wykres: NIG
plt.figure(figsize=(10, 6))
for n in sample_sizes:
    plt.plot([str(p) for p in nig_params], ad_power_nig[n], marker='o', label=f'n={n}')
plt.title('Moc testu Andersona-Darlinga – dane z rozkładu NIG')
plt.xlabel('Parametry rozkładu NIG (α, β, loc, scale)')
plt.ylabel('Moc testu')
plt.ylim(0.0, 1.0)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# === WNIOSKI: Test Andersona-Darlinga – rozkłady gamma i NIG ===
# ▸ Test AD skutecznie wykrywa nienormalność danych z rozkładów gamma i NIG.
# ▸ GAMMA:
#    - Największa moc dla silnie skośnych rozkładów (np. gamma(2,2)).
#    - Wraz ze wzrostem liczebności próby moc testu istotnie rośnie (dla n=100 blisko 1).
# ▸ NIG:
#    - Moc testu również rośnie z licznością próbki, ale jest niższa niż dla gamma.
#    - Im większy parametr α (rozciągnięcie ogonów), tym lepiej test wykrywa odstępstwa od normalności.
# ▸ Test AD jest bardzo czuły na asymetrię i ciężkie ogony – dobrze sprawdza się przy danych, które tylko z pozoru przypominają normalne.
